In [ ]:
from collections import deque
import numpy as np
import pandas as pd
import random


class SequentialOrganAllocationMDP:
    def __init__(self, initial_state, available_organs):
        """
        Initialize the sequential MDP for organ allocation.

        Parameters:
        - initial_state: List of dictionaries representing waitlist candidates.
        - available_organs: Dictionary with available organs by blood type.
        """
        self.initial_state = (
            tuple((r['id'], r['age'], r['MELD'], r['blood_type'], r['allocated']) for r in initial_state),
            tuple(available_organs.items())
        )
        self.value_table = {self.initial_state: 0}  # Value function initialized for the initial state
        self.policy = {self.initial_state: None}  # Policy initialized for the initial state
        self.reward_table = self._precompute_rewards(initial_state)  # Precompute rewards
        self.deltas = []  # To store deltas over iterations

    def _precompute_rewards(self, recipients):
        """
        Precompute rewards for all recipients.

        Parameters:
        - recipients: List of dictionaries representing recipients.

        Returns:
        - A dictionary mapping recipient IDs to precomputed rewards.
        """
        ids = np.array([r['id'] for r in recipients])
        ages = np.array([r['age'] for r in recipients])
        melds = np.array([r['MELD'] for r in recipients])

        rewards = 10 + (1 / (1 + np.maximum(melds, 0))) * 10 - (ages * 0.1)
        reward_table = dict(zip(ids, rewards))

        return reward_table
    
    def calculate_reward(self, recipient):
        """
        Retrieve precomputed reward for the recipient.

        Parameters:
        - recipient: Tuple representing the recipient receiving the organ.

        Returns:
        - Precomputed reward value.
        """
        recipient_id = recipient[0]  # Extract the recipient ID
        return self.reward_table.get(recipient_id, 0)  # Default to 0 if not found

    def generate_next_state(self, state, action):
        """
        Generate the next state based on the current state and action.

        Parameters:
        - state: Tuple (recipients, available_organs).
        - action: Tuple (recipient_id, organ_type).

        Returns:
        - next_state: Updated state after taking the action.
        - reward: Associated reward for the action.
        """
        recipients, available_organs = state
        #print(recipients)
        recipients = list(recipients)
        available_organs = dict(available_organs)

        if action is None:
            return (tuple(recipients), tuple(available_organs.items())), 0

        recipient_id, organ_type = action
        recipient_idx = next((i for i, r in enumerate(recipients) if r[0] == recipient_id), None)

        if recipient_idx is not None and available_organs[organ_type] > 0:

            #Don't allocate organs unless a blood match - can change this slightly as an experiment (what blood can take what blood)
            if recipients[recipient_idx][3] == organ_type:
                if random.random() < 1.1:  # 90% success
                    recipient = recipients[recipient_idx]
                    recipients[recipient_idx] = (recipient[0], recipient[1], recipient[2], recipient[3], 1)
                    available_organs[organ_type] -= 1
                    reward = self.calculate_reward(recipient)
                    #print(f"Action success: Recipient {recipient_id} allocated organ {organ_type}. Reward: {reward}")
                    return (tuple(recipients), tuple(available_organs.items())), reward
                else:  # 10% failure (recipient dies, organ is removed)
                    recipient = recipients.pop(recipient_idx)
                    available_organs[organ_type] -= 1
                    reward = -100  # Large penalty for death
                    #print(f"Action failure: Recipient {recipient_id} died. Organ {organ_type} removed. Reward: {reward}")
                    return (tuple(recipients), tuple(available_organs.items())), reward
        #print(f"Invalid action: No organ {organ_type} available or recipient {recipient_id} not found.")
        return (tuple(recipients), tuple(available_organs.items())), 0
    
    def value_iteration(self, gamma=0.9, epsilon=0.01, max_states=5000):
        """
        Perform value iteration with pruning.
        
        Parameters:
        - gamma: Discount factor (default 0.9).
        - epsilon: Convergence threshold (default 0.01).
        - max_states: Maximum number of states to retain in the queue during pruning.
        """
        states_to_explore = deque([self.initial_state])
        visited_states = set()  # Track visited states
        self.deltas = []  # Reset deltas at the start of value iteration
        iteration_count = 0

        while True:
            iteration_count += 1
            delta = 0
            new_states_to_explore = []

            print(f"Iteration {iteration_count}, States to explore: {len(states_to_explore)}")

            while states_to_explore:
                current_state = states_to_explore.popleft()

                # Skip already visited states
                if current_state in visited_states:
                    continue

                visited_states.add(current_state)
                old_value = self.value_table[current_state]
                max_value = float('-inf')
                best_action = None
                no_valid_actions = True

                recipients, available_organs = current_state
                available_organs = dict(available_organs)

                for recipient in recipients:
                    if recipient[4] == 0:  # Not yet allocated
                        for organ_type in available_organs.keys():
                            if available_organs[organ_type] > 0:
                                no_valid_actions = False
                                action = (recipient[0], organ_type)
                                next_state, reward = self.generate_next_state(current_state, action)
                                value = reward + gamma * self.value_table.get(next_state, 0)

                                if value > max_value:
                                    max_value = value
                                    best_action = action

                                if next_state not in self.value_table:
                                    self.value_table[next_state] = 0
                                    self.policy[next_state] = None
                                    new_states_to_explore.append((next_state, abs(old_value - max_value)))
                if no_valid_actions:
                    continue
                self.value_table[current_state] = max_value
                
                self.policy[current_state] = best_action
                delta = max(delta, abs(old_value - max_value))

            # Prune new states
        
            new_states_to_explore.sort(key=lambda x: x[1], reverse=True)  # Sort by delta (value change)
            new_states_to_explore = [state for state, _ in new_states_to_explore[:max_states]]

            print(f"Pruned to {len(new_states_to_explore)} states.")
            states_to_explore = deque(new_states_to_explore)
            self.deltas.append(delta)
            print(f"Delta: {delta}")

            if delta < epsilon and not states_to_explore:
                break

    def get_deltas(self):
        """
        Retrieve the deltas recorded during value iteration.

        Returns:
        - List of delta values for each iteration.
        """
        return self.deltas #To use for plotting later
    
    def get_max_value(self):
        """
        Retrieve the deltas recorded during value iteration.

        Returns:
        - List of delta values for each iteration.
        """
        return self.max_values #To use for plotting later

    def simulate_with_policy(self, steps=10):
        """
        Simulate the allocation process using the computed policy.

        Parameters:
        - steps: Number of allocation steps to simulate (default 10).

        Returns:
        - Total reward, total deaths, total allocations.
        """
        current_state = self.initial_state
        total_reward = 0
        total_deaths = 0
        total_allocations = 0

        for _ in range(steps):
            action = self.policy.get(current_state, None)
            print(action)
            if action is None:
                break

            next_state, reward = self.generate_next_state(current_state, action)

            if reward == -100:  # Death penalty
                total_deaths += 1
            elif reward > 0:  # Successful allocation
                total_allocations += 1

            total_reward += reward
            current_state = next_state

        return total_reward, total_deaths, total_allocations


# Reinitialize the example with a reduced state space for debugging
df = pd.read_csv('sampled_waitlist.csv')
np.random.seed(42)
df = df.iloc[np.random.choice(np.arange(0, len(df)), size=50, replace=False)] #Take a small sample 
initial_state = df.apply(
    lambda row: {
        'id': row.name + 1,  # Generate an 'id' starting from 1
        'age': row['RECIPIENT_AGE'],  # Replace with the actual age column if available
        'MELD': row['INIT_MELD_PELD_LAB_SCORE'],
        'blood_type': row['RECIPIENT_BLOOD_TYPE'],
        'allocated': 0  # Default value
    }, axis=1
).tolist()
print(len(initial_state))
available_organs = {'A': 2, 'A1': 3, 'A1B': 1, 'A2': 1, 'A2B': 1, 'AB': 1, 'B': 2, 'O': 7, 'AB': 1}
'''
initial_state = [
    {'id': i, 'age': random.randint(20, 70), 'MELD': random.randint(10, 40), 'blood_type': random.choice(['A', 'B', 'O', 'AB']), 'allocated': 0}
    for i in range(1, 6)
]
initial_state = [{'id': 1, 'age': 70, 'MELD': 33, 'blood_type': 'O', 'allocated': 0},
 {'id': 2, 'age': 51, 'MELD': 17, 'blood_type': 'O', 'allocated': 0},
 {'id': 3, 'age': 24, 'MELD': 10, 'blood_type': 'AB', 'allocated': 0},
 {'id': 4, 'age': 50, 'MELD': 38, 'blood_type': 'AB', 'allocated': 0},
 {'id': 5, 'age': 23, 'MELD': 26, 'blood_type': 'O', 'allocated': 0}]
available_organs = {'A': 1, 'B': 1, 'O': 1, 'AB': 1}
'''




50
Iteration 1, States to explore: 1
Pruned to 5 states.
Delta: 8.50909090909091
Iteration 2, States to explore: 5
Pruned to 6 states.
Delta: 8.50909090909091
Iteration 3, States to explore: 6
Pruned to 0 states.
Delta: 0.0
(3, 'AB')
(5, 'O')
(1, 'A')
(1, 'A')
(1, 'A')


(16.579461279461277, 0, 2)

Timer unit: 1e-07 s

Total time: 0.0060228 s
File: C:\Users\Veronica\AppData\Local\Temp\ipykernel_38212\3864078132.py
Function: value_iteration at line 100

Line #      Hits         Time  Per Hit   % Time  Line Contents
   100                                               def value_iteration(self, gamma=0.9, epsilon=0.01, max_states=5000):
   101                                                   """
   102                                                   Perform value iteration with pruning.
   103                                                   
   104                                                   Parameters:
   105                                                   - gamma: Discount factor (default 0.9).
   106                                                   - epsilon: Convergence threshold (default 0.01).
   107                                                   - max_states: Maximum number of states to retain in the queue during pruning.
   108                               

In [13]:
%load_ext line_profiler

In [15]:
df = pd.read_csv('sampled_waitlist.csv')
np.random.seed(42)
df = df.iloc[np.random.choice(np.arange(0, len(df)), size=50, replace=False)] #Take a small sample 
initial_state = df.apply(
    lambda row: {
        'id': row.name + 1,  # Generate an 'id' starting from 1
        'age': row['RECIPIENT_AGE'],  # Replace with the actual age column if available
        'MELD': row['INIT_MELD_PELD_LAB_SCORE'],
        'blood_type': row['RECIPIENT_BLOOD_TYPE'],
        'allocated': 0  # Default value
    }, axis=1
).tolist()
print(len(initial_state))
available_organs = {'A': 2, 'A1': 3, 'A1B': 1, 'A2': 1, 'A2B': 1, 'AB': 1, 'B': 2, 'O': 7, 'AB': 1}

# Initialize and execute the MDP
mdp_model = SequentialOrganAllocationMDP(initial_state, available_organs)
#%prun mdp_model.value_iteration(gamma=0.9, epsilon=0.01, max_states=5000)
#mdp_model.value_iteration(gamma=0.9, epsilon=0.01, max_states=1000)
%lprun -f mdp_model.value_iteration mdp_model.value_iteration(gamma=0.9, epsilon=0.01, max_states=1000)

deltas = mdp_model.get_deltas()
total_reward, total_deaths, total_allocations = mdp_model.simulate_with_policy(steps=len(initial_state))


total_reward, total_deaths, total_allocations

50
Iteration 1, States to explore: 1
Pruned to 50 states.
Delta: 8.694117647058825
Iteration 2, States to explore: 50
Pruned to 1000 states.
Delta: 8.694117647058825
Iteration 3, States to explore: 1000
Pruned to 1000 states.
Delta: 8.694117647058825
Iteration 4, States to explore: 1000
Pruned to 1000 states.
Delta: 8.694117647058825
Iteration 5, States to explore: 1000
Pruned to 1000 states.
Delta: 8.694117647058825
Iteration 6, States to explore: 1000
Pruned to 1000 states.
Delta: 8.694117647058825
Iteration 7, States to explore: 1000
Pruned to 1000 states.
Delta: 8.694117647058825
Iteration 8, States to explore: 1000
Pruned to 1000 states.
Delta: 8.694117647058825
Iteration 9, States to explore: 1000
Pruned to 1000 states.
Delta: 8.694117647058825
Iteration 10, States to explore: 1000
Pruned to 1000 states.
Delta: 7.133333333333334
Iteration 11, States to explore: 1000
Pruned to 1000 states.
Delta: 7.133333333333334
Iteration 12, States to explore: 1000
Pruned to 1000 states.
Delta:

(22.803641456582632, 0, 3)

Timer unit: 1e-07 s

Total time: 74.9352 s
File: C:\Users\Veronica\AppData\Local\Temp\ipykernel_38212\518608690.py
Function: value_iteration at line 81

Line #      Hits         Time  Per Hit   % Time  Line Contents
    81                                               def value_iteration(self, gamma=0.9, epsilon=0.01, max_states=5000):
    82                                                   """
    83                                                   Perform value iteration with pruning.
    84                                                   
    85                                                   Parameters:
    86                                                   - gamma: Discount factor (default 0.9).
    87                                                   - epsilon: Convergence threshold (default 0.01).
    88                                                   - max_states: Maximum number of states to retain in the queue during pruning.
    89                                   

In [31]:
import pstats

stats = pstats.Stats('profile_output')
stats.strip_dirs().sort_stats('tottime').print_stats(100)

Tue Dec  3 21:37:21 2024    profile_output

         193994214 function calls (193982512 primitive calls) in 669.875 seconds

   Ordered by: internal time
   List reduced from 2743 to 100 due to restriction <100>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1  216.965  216.965  667.260  667.260 Value_Iteration.py:102(value_iteration)
 34003823  171.049    0.000  294.830    0.000 Value_Iteration.py:59(generate_next_state)
 37484451  152.067    0.000  152.067    0.000 {method 'get' of 'dict' objects}
 34003823  102.272    0.000  102.272    0.000 Value_Iteration.py:80(<genexpr>)
 34003890    9.912    0.000  112.183    0.000 {built-in method builtins.next}
 34004205    7.905    0.000    7.905    0.000 {method 'items' of 'dict' objects}
        1    1.892    1.892  669.876  669.876 Value_Iteration.py:1(<module>)
  3469573    1.578    0.000    2.806    0.000 Value_Iteration.py:46(calculate_reward)
  5115719    1.453    0.000    1.453    0.000 {method 'keys

In [18]:
df = pd.read_csv('sampled_waitlist.csv')
np.random.seed(42)
df = df.iloc[np.random.choice(np.arange(0, len(df)), size=50, replace=False)] #Take a small sample 
initial_state = df.apply(
    lambda row: {
        'id': row.name + 1,  # Generate an 'id' starting from 1
        'age': row['RECIPIENT_AGE'],  # Replace with the actual age column if available
        'MELD': row['INIT_MELD_PELD_LAB_SCORE'],
        'blood_type': row['RECIPIENT_BLOOD_TYPE'],
        'allocated': 0  # Default value
    }, axis=1
).tolist()
print(len(initial_state))
available_organs = {'A': 2, 'A1': 3, 'A1B': 1, 'A2': 1, 'A2B': 1, 'AB': 1, 'B': 2, 'O': 7, 'AB': 1}

# Initialize and execute the MDP
mdp_model = SequentialOrganAllocationMDP(initial_state, available_organs)
#%prun mdp_model.value_iteration(gamma=0.9, epsilon=0.01, max_states=5000)
#mdp_model.value_iteration(gamma=0.9, epsilon=0.01, max_states=1000)
%lprun -f mdp_model.value_iteration mdp_model.value_iteration(gamma=0.9, epsilon=0.01, max_states=1000)

deltas = mdp_model.get_deltas()
total_reward, total_deaths, total_allocations = mdp_model.simulate_with_policy(steps=len(initial_state))


total_reward, total_deaths, total_allocations

50
Iteration 1, States to explore: 1
Pruned to 50 states.
Delta: 8.694117647058825
Iteration 2, States to explore: 50
Pruned to 1000 states.
Delta: 8.694117647058825
Iteration 3, States to explore: 1000
Pruned to 1000 states.
Delta: 8.694117647058825
Iteration 4, States to explore: 1000
Pruned to 1000 states.
Delta: 8.694117647058825
Iteration 5, States to explore: 1000
Pruned to 1000 states.
Delta: 8.694117647058825
Iteration 6, States to explore: 1000
Pruned to 1000 states.
Delta: 8.694117647058825
Iteration 7, States to explore: 1000
Pruned to 1000 states.
Delta: 8.694117647058825
Iteration 8, States to explore: 1000
Pruned to 1000 states.
Delta: 8.694117647058825
Iteration 9, States to explore: 1000
Pruned to 1000 states.
Delta: 8.694117647058825
Iteration 10, States to explore: 1000
Pruned to 1000 states.
Delta: 8.694117647058825
Iteration 11, States to explore: 1000
Pruned to 1000 states.
Delta: 8.694117647058825
Iteration 12, States to explore: 1000
Pruned to 1000 states.
Delta:

(22.803641456582632, 0, 3)

Timer unit: 1e-07 s

Total time: 74.4985 s
File: C:\Users\Veronica\AppData\Local\Temp\ipykernel_38212\373946340.py
Function: value_iteration at line 100

Line #      Hits         Time  Per Hit   % Time  Line Contents
   100                                               def value_iteration(self, gamma=0.9, epsilon=0.01, max_states=5000):
   101                                                   """
   102                                                   Perform value iteration with pruning.
   103                                                   
   104                                                   Parameters:
   105                                                   - gamma: Discount factor (default 0.9).
   106                                                   - epsilon: Convergence threshold (default 0.01).
   107                                                   - max_states: Maximum number of states to retain in the queue during pruning.
   108                                  

In [7]:
initial_state

[{'id': 362, 'age': 35, 'MELD': 20.0, 'blood_type': 'O', 'allocated': 0},
 {'id': 74, 'age': 65, 'MELD': 33.0, 'blood_type': 'AB', 'allocated': 0},
 {'id': 375, 'age': 72, 'MELD': 26.0, 'blood_type': 'O', 'allocated': 0},
 {'id': 156, 'age': 53, 'MELD': 21.0, 'blood_type': 'B', 'allocated': 0},
 {'id': 105, 'age': 72, 'MELD': 13.0, 'blood_type': 'B', 'allocated': 0},
 {'id': 395, 'age': 49, 'MELD': 34.0, 'blood_type': 'O', 'allocated': 0},
 {'id': 378, 'age': 45, 'MELD': 27.0, 'blood_type': 'O', 'allocated': 0},
 {'id': 125, 'age': 50, 'MELD': 16.0, 'blood_type': 'B', 'allocated': 0},
 {'id': 69, 'age': 62, 'MELD': 24.0, 'blood_type': 'AB', 'allocated': 0},
 {'id': 451, 'age': 45, 'MELD': 37.0, 'blood_type': 'O', 'allocated': 0},
 {'id': 10, 'age': 64, 'MELD': 8.0, 'blood_type': 'A', 'allocated': 0},
 {'id': 195, 'age': 46, 'MELD': 30.0, 'blood_type': 'O', 'allocated': 0},
 {'id': 407, 'age': 50, 'MELD': 24.0, 'blood_type': 'O', 'allocated': 0},
 {'id': 85, 'age': 63, 'MELD': 17.0, 'bl

In [68]:
np.random.seed(42)

# Generate 10 unique random numbers between 1 and 100
unique_random_numbers = np.random.choice(np.arange(0, len(df)), size=20, replace=False)

df.iloc[np.random.choice(np.arange(0, len(df)), size=30, replace=False)]

,RECIPIENT_AGE,INIT_MELD_PELD_LAB_SCORE,RECIPIENT_BLOOD_TYPE,GRAFT_LIFESPAN
306,36,45.0,O,198.0
449,31,28.0,O,369.0
123,74,7.0,B,72.0
194,46,30.0,O,6.0
66,60,38.0,A,63.0
394,49,34.0,O,184.0
430,51,20.0,O,377.0
345,57,17.0,O,211.0
498,51,17.0,O,15.0
237,66,14.0,O,447.0


In [6]:
initial_state = [{'id': 1, 'age': 70, 'MELD': 33, 'blood_type': 'O', 'allocated': 0},
 {'id': 2, 'age': 51, 'MELD': 17, 'blood_type': 'O', 'allocated': 0},
 {'id': 3, 'age': 24, 'MELD': 10, 'blood_type': 'AB', 'allocated': 0},
 {'id': 4, 'age': 50, 'MELD': 38, 'blood_type': 'AB', 'allocated': 0},
 {'id': 5, 'age': 23, 'MELD': 26, 'blood_type': 'O', 'allocated': 0}]
initial_state

[{'id': 1, 'age': 70, 'MELD': 33, 'blood_type': 'O', 'allocated': 0},
 {'id': 2, 'age': 51, 'MELD': 17, 'blood_type': 'O', 'allocated': 0},
 {'id': 3, 'age': 24, 'MELD': 10, 'blood_type': 'AB', 'allocated': 0},
 {'id': 4, 'age': 50, 'MELD': 38, 'blood_type': 'AB', 'allocated': 0},
 {'id': 5, 'age': 23, 'MELD': 26, 'blood_type': 'O', 'allocated': 0}]

In [10]:
recipient = initial_state[0]
recipient

{'id': 1, 'age': 70, 'MELD': 33, 'blood_type': 'O', 'allocated': 0}

In [34]:
organs = pd.read_csv('available_organs.csv')
valid = organs[organs['INIT_MELD_OR_PELD'] == 'MELD'].reset_index(drop=True)

valid.head()

,ORGAN_TRANSPLANT_ID,WAITLIST_ID,RECIPIENT_ID,DONOR_ID,RECIPIENT_GENDER,RECIPIENT_AGE,DONOR_AGE,INIT_MELD_OR_PELD,INIT_MELD_PELD_LAB_SCORE,FINAL_MELD_OR_PELD,...,DONOR_RECIPIENT_BLOOD_LEVEL,GRAFT_FUNCTIONING,GRAFT_LIFESPAN,DONOR_ADMISSION_DATE,DONOR_TYPE,ORGAN_RECOVERY_DATE,WAITLIST_REGISTRATION_DATE,END_DATE,REASON_REMOVED_WAITLIST,RECIPIENT_STATUS
0,A903126,1575801,1384194,617432,M,68,83.0,MELD,7.0,MELD,...,1.0,N,307.0,08/25/2021,DECEASED,2021-09-08,2021-01-20,2021-09-08,TRANSPLANT,DIED
1,A527822,919869,830133,353347,M,67,73.0,MELD,6.0,MELD,...,1.0,Y,4.0,06/18/2010,DECEASED,2010-06-24,2010-05-11,2010-06-24,TRANSPLANT,DIED
2,A940792,1650700,1448221,642256,F,56,46.0,MELD,6.0,MELD,...,3.0,N,561.0,08/06/2022,DECEASED,2022-08-09,2022-02-21,2022-08-10,TRANSPLANT,DIED
3,A680595,1117735,999468,478649,M,64,41.0,MELD,9.0,MELD,...,1.0,Y,3281.0,09/11/2015,DECEASED,2015-09-13,2013-09-30,2015-09-13,TRANSPLANT,ALIVE
4,A702528,1274856,1132326,493817,F,52,33.0,MELD,30.0,MELD,...,1.0,Y,2898.0,05/21/2016,DECEASED,2016-05-26,2016-05-12,2016-05-26,TRANSPLANT,ALIVE


In [35]:
df = organs[organs['DONOR_ADMISSION_DATE'] >= '01/01/2021'].reset_index(drop=True)
df[['DONOR_ADMISSION_DATE', 'DONOR_BLOOD_TYPE']].groupby('DONOR_ADMISSION_DATE').value_counts().groupby('DONOR_BLOOD_TYPE').describe()

,count,mean,std,min,25%,50%,75%,max
DONOR_BLOOD_TYPE,,,,,,,,
A,5666.0,2.663431,1.662587,1.0,1.0,2.0,4.0,14.0
A1,6227.0,3.237675,1.748123,1.0,2.0,3.0,4.0,11.0
A1B,1132.0,1.093640,0.317587,1.0,1.0,1.0,1.0,4.0
A2,2718.0,1.368653,0.676818,1.0,1.0,1.0,2.0,6.0
A2B,505.0,1.039604,0.195220,1.0,1.0,1.0,1.0,2.0
AB,1247.0,1.115477,0.341570,1.0,1.0,1.0,1.0,4.0
B,5436.0,2.288263,1.323435,1.0,1.0,2.0,3.0,10.0
O,6707.0,7.653496,3.769704,1.0,5.0,7.0,10.0,27.0


In [67]:
valid[['WAITLIST_REGISTRATION_DATE', 'RECIPIENT_BLOOD_TYPE']].groupby('WAITLIST_REGISTRATION_DATE').value_counts().groupby('RECIPIENT_BLOOD_TYPE').describe()

,count,mean,std,min,25%,50%,75%,max
RECIPIENT_BLOOD_TYPE,,,,,,,,
A,6725.0,6.171301,3.974387,1.0,3.0,6.0,9.0,26.0
A1,161.0,1.043478,0.233126,1.0,1.0,1.0,1.0,3.0
A1B,12.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
A2,24.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
A2B,9.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
AB,3630.0,1.601102,0.890907,1.0,1.0,1.0,2.0,7.0
B,5597.0,2.760050,1.772272,1.0,1.0,2.0,4.0,12.0
O,6917.0,7.329912,4.872183,1.0,3.0,7.0,10.0,30.0


In [69]:
set(organs['REASON_REMOVED_WAITLIST'])

{'DIED_DURING_TRANSPLANT', 'TRANSPLANT'}

In [181]:
import pandas as pd


df = valid[valid['WAITLIST_REGISTRATION_DATE'] >= pd.to_datetime('2021-01-01')].reset_index(drop=True)

# Ensure dates are in datetime format
df['WAITLIST_REGISTRATION_DATE'] = pd.to_datetime(df['WAITLIST_REGISTRATION_DATE'])
df['END_DATE'] = pd.to_datetime(df['END_DATE'])

# Create a date range for each row
df['DATE_RANGE'] = df.apply(
    lambda row: pd.date_range(row['WAITLIST_REGISTRATION_DATE'], row['END_DATE']), axis=1
)

# Explode the date range into individual rows
df_exploded = df.explode('DATE_RANGE')

df_exploded[['DATE_RANGE', 'RECIPIENT_BLOOD_TYPE']].groupby('DATE_RANGE')['RECIPIENT_BLOOD_TYPE'].value_counts().groupby('RECIPIENT_BLOOD_TYPE').describe()
#This is how many people are on the waitlist each day

,count,mean,std,min,25%,50%,75%,max
RECIPIENT_BLOOD_TYPE,,,,,,,,
A,1369.0,688.142440,236.784890,1.0,567.0,778.0,868.0,940.0
A1,1003.0,1.660020,0.654378,1.0,1.0,2.0,2.0,4.0
A1B,4.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
A2,148.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
A2B,27.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
AB,1367.0,59.953914,18.127701,1.0,50.0,65.0,73.0,96.0
B,1369.0,235.149014,75.019830,1.0,199.0,262.0,289.0,326.0
O,1369.0,890.037253,322.349350,2.0,691.0,1015.0,1160.0,1244.0


In [182]:
list(df_exploded[['DATE_RANGE', 'RECIPIENT_BLOOD_TYPE']].groupby('DATE_RANGE')['RECIPIENT_BLOOD_TYPE'].value_counts().groupby('RECIPIENT_BLOOD_TYPE').median())

[778.0, 2.0, 1.0, 1.0, 1.0, 65.0, 262.0, 1015.0]

In [116]:
#Grab right data (last 2 year avg) - t
df = valid[valid['WAITLIST_REGISTRATION_DATE'] >= pd.to_datetime('2022-01-01')].reset_index(drop=True)
df = df[['RECIPIENT_AGE', 'INIT_MELD_PELD_LAB_SCORE', 'RECIPIENT_BLOOD_TYPE',
       'GRAFT_LIFESPAN']]

In [201]:
#Sample from dirchlet
data = {
    'RECIPIENT_BLOOD_TYPE': ['A', 'A1', 'A1B', 'A2', 'A2B', 'AB', 'B', 'O'],
    'Median': [778.0, 2.0, 1.0, 1.0, 1.0, 65.0, 262.0, 1015.0]
}

proportion_df = pd.DataFrame(data)
proportion_df['Proportion'] = proportion_df['Median'] / proportion_df['Median'].sum()

alpha = proportion_df['Proportion'].values *10 #Accounting for some variance and to ensure get sample of lowerp proportion blood types

np.random.seed(123)
dirichlet_sample = np.random.dirichlet(alpha, size=1).flatten()

#Sample size 500
sample_counts = (dirichlet_sample * 500).round().astype(int)

# Add sample counts to the proportion DataFrame
proportion_df['SampleCount'] = sample_counts

df = valid[valid['WAITLIST_REGISTRATION_DATE'] >= pd.to_datetime('2022-01-01')].reset_index(drop=True)
df = df[['RECIPIENT_AGE', 'INIT_MELD_PELD_LAB_SCORE', 'RECIPIENT_BLOOD_TYPE',
       'GRAFT_LIFESPAN']]

# Step 5: Perform stratified sampling based on Dirichlet-generated counts
sampled_data = pd.DataFrame()
for blood_type, count in zip(proportion_df['RECIPIENT_BLOOD_TYPE'], proportion_df['SampleCount']):
    # Filter rows matching the blood type
    group = df[df['RECIPIENT_BLOOD_TYPE'] == blood_type]
    # Sample the calculated number of rows
    sampled_group = group.sample(n=min(count, len(group)), random_state=42, replace=True)
    sampled_data = pd.concat([sampled_data, sampled_group])

# Step 6: Reset index on the sampled DataFrame
sampled_data = sampled_data.reset_index(drop=True)
sampled_data.to_csv('sampled_waitlist.csv', index=False)

# Output the sampled data
sampled_data.groupby('RECIPIENT_BLOOD_TYPE').count()


,RECIPIENT_AGE,INIT_MELD_PELD_LAB_SCORE,GRAFT_LIFESPAN
RECIPIENT_BLOOD_TYPE,,,
A,68,68,64
AB,14,14,13
B,87,87,83
O,331,331,315


In [ ]:
df[['DONOR_ADMISSION_DATE', 'DONOR_BLOOD_TYPE']].groupby('DONOR_ADMISSION_DATE').value_counts().groupby('DONOR_BLOOD_TYPE').median()
data = {
    'DONOR_BLOOD_TYPE': ['A', 'A1', 'A1B', 'A2', 'A2B', 'AB', 'B', 'O'],
    'Median': [2.0, 3.0, 1.0, 1.0, 1.0, 1.0, 2.0, 7.0]
}

In [ ]:
import pandas as pd

# Example: Data structure for reference
data = {
    'RECIPIENT_BLOOD_TYPE': ['A', 'A1', 'A2', 'A2B', 'AB', 'B', 'O'],
    'Median': [507, 2, 1, 1, 50, 199, 638]  # Median values from your table
}

# Step 1: Create a DataFrame from the provided proportions
proportion_df = pd.DataFrame(data)

# Step 2: Calculate proportions of each blood type at the median
total_median = proportion_df['Median'].sum()
proportion_df['Proportion'] = proportion_df['Median'] / total_median

# Step 3: Calculate sampling counts for 500 samples
total_samples = 500
proportion_df['SampleCount'] = (proportion_df['Proportion'] * total_samples).round().astype(int)

# Example: Original DataFrame (Replace with your actual DataFrame)
df = pd.DataFrame({
    'RECIPIENT_BLOOD_TYPE': ['A', 'A', 'A1', 'A2', 'A2B', 'AB', 'B', 'O'] * 100,  # Simplified example
    'Other_Column': range(800)
})

# Step 4: Perform stratified sampling
sampled_data = pd.DataFrame()
for blood_type, count in zip(proportion_df['RECIPIENT_BLOOD_TYPE'], proportion_df['SampleCount']):
    # Filter rows matching the blood type
    group = df[df['RECIPIENT_BLOOD_TYPE'] == blood_type]
    # Sample the calculated number of rows
    sampled_group = group.sample(n=min(count, len(group)), random_state=42)
    sampled_data = pd.concat([sampled_data, sampled_group])

# Step 5: Reset index on the sampled DataFrame
sampled_data = sampled_data.reset_index(drop=True)

# Output the sampled data
print(sampled_data)


In [96]:
df = pd.read_csv('waitlist_patients.csv')

initial_state = df.apply(
    lambda row: {
        'id': row.name + 1,  # Generate an 'id' starting from 1
        'age': row['RECIPIENT_AGE'],  # Replace with the actual age column if available
        'MELD': row['INIT_MELD_PELD_LAB_SCORE'],
        'blood_type': row['RECIPIENT_BLOOD_TYPE'],
        'allocated': 0  # Default value
    }, axis=1
).tolist()

In [113]:
df = pd.read_csv('waitlist_patients.csv')
df.columns

Index(['RECIPIENT_AGE', 'INIT_MELD_PELD_LAB_SCORE', 'RECIPIENT_BLOOD_TYPE',
       'GRAFT_LIFESPAN'],
      dtype='object')

In [36]:
import pandas as pd
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Number of patients and organs
num_patients = 10
num_organs = 5

# Create fake patient data
patients = pd.DataFrame({
    "patient_id": [f"P{i}" for i in range(1, num_patients + 1)],
    "health_status": np.random.choice(["Critical", "Stable", "Recovered", "Deceased"], num_patients, p=[0.4, 0.4, 0.1, 0.1]),
    "priority_score": np.random.randint(1, 101, num_patients),  # Priority from 1 to 100
    "survival_probability": np.round(np.random.uniform(0.1, 0.9, num_patients), 2)
})

# Create fake organ data
organs = pd.DataFrame({
    "organ_id": [f"O{i}" for i in range(1, num_organs + 1)],
    "compatibility": [np.random.choice(patients["patient_id"], np.random.randint(1, 4), replace=False).tolist() for _ in range(num_organs)],
    "viability_time": np.random.randint(1, 48, num_organs)  # Viability in hours
})

# Create fake transition probabilities
transitions = pd.DataFrame({
    "current_state": np.random.choice(["Critical", "Stable", "Recovered", "Deceased"], 20),
    "action": np.random.choice(["Assign", "Hold", "Discard"], 20),
    "next_state": np.random.choice(["Critical", "Stable", "Recovered", "Deceased"], 20),
    "probability": np.round(np.random.uniform(0.1, 1.0, 20), 2)
})
transitions = transitions.groupby(["current_state", "action"]).apply(lambda x: x.assign(probability=x["probability"] / x["probability"].sum())).reset_index(drop=True)

# Create fake rewards
rewards = pd.DataFrame({
    "action": ["Assign", "Hold", "Discard"],
    "reward": [100, -10, -50]
})




C:\Users\Veronica\AppData\Local\Temp\ipykernel_38212\2441431421.py:33: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  transitions = transitions.groupby(["current_state", "action"]).apply(lambda x: x.assign(probability=x["probability"] / x["probability"].sum())).reset_index(drop=True)


In [ ]:
import numpy as np
import pandas as pd

def value_iteration(states, actions, transition_probabilities, rewards, gamma=0.95, threshold=1e-3):
    """
    Perform value iteration to find the optimal policy.
    
    Parameters:
        states (list): List of states.
        actions (list): List of actions.
        transition_probabilities (DataFrame): DataFrame with columns ['current_state', 'action', 'next_state', 'probability'].
        rewards (DataFrame): DataFrame with columns ['action', 'reward'].
        gamma (float): Discount factor (default 0.95).
        threshold (float): Convergence threshold (default 1e-3).
    
    Returns:
        value_table (dict): Dictionary of state values.
        policy (dict): Dictionary of optimal actions for each state.
    """
    # Initialize value table
    value_table = {state: 0 for state in states}
    
    # Extract reward per action
    reward_dict = dict(zip(rewards["action"], rewards["reward"]))
    
    # Iterate until convergence
    while True:
        delta = 0
        new_value_table = value_table.copy()
        
        for state in states:            
            action_values = {}
            
            for action in actions:
                # Get transition probabilities for this state and action
                transitions = transition_probabilities[
                    (transition_probabilities["current_state"] == state) &
                    (transition_probabilities["action"] == action)
                ]
                
                # Calculate expected value for this action
                expected_value = sum(
                    row["probability"] * (reward_dict[action] + gamma * value_table[row["next_state"]])
                    for _, row in transitions.iterrows()
                )
                action_values[action] = expected_value
            
            # Update value for this state
            new_value_table[state] = max(action_values.values()) if action_values else 0
            
            # Track maximum change for convergence check
            delta = max(delta, abs(new_value_table[state] - value_table[state]))
        
        value_table = new_value_table
        
        if delta < threshold:
            break
    
    # Extract optimal policy
    policy = {}
    for state in states:
        action_values = {}
        
        for action in actions:
            transitions = transition_probabilities[
                (transition_probabilities["current_state"] == state) &
                (transition_probabilities["action"] == action)
            ]
            
            expected_value = sum(
                row["probability"] * (reward_dict[action] + gamma * value_table[row["next_state"]])
                for _, row in transitions.iterrows()
            )
            action_values[action] = expected_value
        
        policy[state] = max(action_values, key=action_values.get) if action_values else None
    
    return value_table, policy

# Extracting states, actions, and transition probabilities
states = transitions["current_state"].unique().tolist()
actions = rewards["action"].tolist()

# Run value iteration
value_table, optimal_policy = value_iteration(states, actions, transitions, rewards)

# Display results
print("Optimal Value Table:")
for state, value in value_table.items():
    print(f"State {state}: {value:.2f}")

print("\nOptimal Policy:")
for state, action in optimal_policy.items():
    print(f"State {state}: {action}")


Critical
Deceased
Recovered
Stable
Critical
Deceased
Recovered
Stable
Critical
Deceased
Recovered
Stable
Critical
Deceased
Recovered
Stable
Critical
Deceased
Recovered
Stable
Critical
Deceased
Recovered
Stable
Critical
Deceased
Recovered
Stable
Critical
Deceased
Recovered
Stable
Critical
Deceased
Recovered
Stable
Critical
Deceased
Recovered
Stable
Critical
Deceased
Recovered
Stable
Critical
Deceased
Recovered
Stable
Critical
Deceased
Recovered
Stable
Critical
Deceased
Recovered
Stable
Critical
Deceased
Recovered
Stable
Critical
Deceased
Recovered
Stable
Critical
Deceased
Recovered
Stable
Critical
Deceased
Recovered
Stable
Critical
Deceased
Recovered
Stable
Critical
Deceased
Recovered
Stable
Critical
Deceased
Recovered
Stable
Critical
Deceased
Recovered
Stable
Critical
Deceased
Recovered
Stable
Critical
Deceased
Recovered
Stable
Critical
Deceased
Recovered
Stable
Critical
Deceased
Recovered
Stable
Critical
Deceased
Recovered
Stable
Critical
Deceased
Recovered
Stable
Critical
Deceased
Re

In [39]:
import pandas as pd
import numpy as np

# Simulated state
current_state = {
    "critical_patients": patients[patients["health_status"] == "Critical"]["patient_id"].tolist(),
    "stable_patients": patients[patients["health_status"] == "Stable"]["patient_id"].tolist(),
    "available_organs": organs["organ_id"].tolist()
}

def choose_patient(policy, current_state, patients, organs):
    """
    Select the best patient for organ allocation based on the policy.
    
    Parameters:
        policy (dict): Optimal policy mapping states to actions.
        current_state (dict): Current state with patient and organ availability.
        patients (DataFrame): DataFrame containing patient details.
        organs (DataFrame): DataFrame containing organ details.
    
    Returns:
        allocation (dict): Dictionary indicating which organ is allocated to which patient.
    """
    allocation = {}
    for organ in current_state["available_organs"]:
        # Get compatible patients for this organ
        compatible_patients = organs[organs["organ_id"] == organ]["compatibility"].values[0]
        
        # Filter out deceased patients
        viable_patients = [
            patient for patient in compatible_patients
            if patient in patients[patients["health_status"] != "Deceased"]["patient_id"].tolist()
        ]
        
        if not viable_patients:
            # No viable patients for this organ
            allocation[organ] = "Discard"
            continue
        
        # Evaluate the best patient based on priority score and survival probability
        best_patient = max(
            viable_patients,
            key=lambda pid: patients[patients["patient_id"] == pid]["priority_score"].values[0] +
                            patients[patients["patient_id"] == pid]["survival_probability"].values[0]
        )
        
        # Assign organ to the best patient
        allocation[organ] = best_patient
    
    return allocation

# Generate optimal policy using value iteration
states = transitions["current_state"].unique().tolist()
actions = rewards["action"].tolist()
value_table, optimal_policy = value_iteration(states, actions, transitions, rewards)

# Use the policy to allocate organs
allocation = choose_patient(optimal_policy, current_state, patients, organs)

# Display the results
print("Organ Allocation Decisions:")
for organ, patient in allocation.items():
    if patient == "Discard":
        print(f"Organ {organ} was discarded (no viable patients).")
    else:
        print(f"Organ {organ} was allocated to patient {patient}.")


Organ Allocation Decisions:
Organ O1 was allocated to patient P9.
Organ O2 was allocated to patient P3.
Organ O3 was allocated to patient P10.
Organ O4 was allocated to patient P10.
Organ O5 was allocated to patient P8.


In [40]:
import numpy as np
import pandas as pd

def transition_system_state(current_state, allocation, patients, transition_probabilities):
    """
    Simulate a transition to the next system state based on organ allocation and patient probabilities.
    
    Parameters:
        current_state (dict): The current state of the system (patients and organs).
        allocation (dict): The organ allocation decisions (organ -> patient or "Discard").
        patients (DataFrame): Patient data with survival probabilities.
        transition_probabilities (DataFrame): Transition probabilities for individual patients.
    
    Returns:
        next_state (dict): The next system state after transitions.
    """
    next_state = {"patients": {}, "organs_available": current_state["available_organs"]}
    
    # Transition each patient based on allocation and probabilities
    for patient_id in patients["patient_id"]:
        if patient_id in allocation.values():
            # Patient receives an organ; transition to "Recovered" or other states
            patient_transitions = transition_probabilities[
                (transition_probabilities["current_state"] == patients.loc[patients["patient_id"] == patient_id, "health_status"].values[0]) &
                (transition_probabilities["action"] == "Assign")
            ]
        else:
            # Patient doesn't receive an organ; transition based on "Hold"
            patient_transitions = transition_probabilities[
                (transition_probabilities["current_state"] == patients.loc[patients["patient_id"] == patient_id, "health_status"].values[0]) &
                (transition_probabilities["action"] == "Hold")
            ]
        
        # Sample the next state for this patient based on transition probabilities
        states = patient_transitions["next_state"].tolist()
        probabilities = patient_transitions["probability"].tolist()
        next_state["patients"][patient_id] = np.random.choice(states, p=probabilities)
    
    # Update organ availability
    next_state["organs_available"] = [
        organ for organ in current_state["available_organs"]
        if allocation.get(organ, "Discard") == "Discard"
    ]
    
    return next_state

# Example usage
current_state = {
    "patients": {patient_id: status for patient_id, status in zip(patients["patient_id"], patients["health_status"])},
    "available_organs": organs["organ_id"].tolist(),
}

# Example allocation (from previous code)
allocation = choose_patient(optimal_policy, current_state, patients, organs)

# Simulate a transition
next_state = transition_system_state(current_state, allocation, patients, transitions)
print("Next State:", next_state)


ValueError: 'a' cannot be empty unless no samples are taken